In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
df = pd.read_csv('steam_reviews.csv')

In [3]:
# dataframe = df.sample(100000, random_state=42)
dataframe = df
dataframe['review'] = dataframe['review'].str.replace('\d+', '') #remove digits
dataframe['review'] = dataframe['review'].str.replace(r'(\b\w{1,2}\b)', '') #remove less than 3 letters words
dataframe['review'] = dataframe['review'].str.replace('[^\w\s]', '') #remove punctuations
body = np.array(dataframe['review'].astype(str))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [4]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer 

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

vec = CountVectorizer(tokenizer=LemmaTokenizer(), strip_accents='ascii', stop_words='english')
bag_of_words = vec.fit_transform(body)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


In [5]:
n_components = 10
svd = TruncatedSVD(n_components=n_components)
lsa = svd.fit_transform(bag_of_words)

In [6]:
columns = [f'Topic_{i}'for i in range(n_components)]
x = ['body']
x.extend(columns)
x

['body',
 'Topic_0',
 'Topic_1',
 'Topic_2',
 'Topic_3',
 'Topic_4',
 'Topic_5',
 'Topic_6',
 'Topic_7',
 'Topic_8',
 'Topic_9']

In [7]:
topic_df = pd.DataFrame(lsa, columns = columns)
topic_df["body"] = body

In [8]:
dictionary = vec.get_feature_names_out()

In [9]:
enconding_matrix = pd.DataFrame(svd.components_,
                                index=columns,
                                columns=dictionary).T

In [10]:
for topic in columns:
    enconding_matrix[f'abs_{topic}'] = np.abs(enconding_matrix[topic])

In [11]:
for topic in columns:
    abs_column = f'abs_{topic}'
    print('\n\n', abs_column)
    print(enconding_matrix.sort_values(abs_column, ascending=False).head(5))



 abs_Topic_0
       Topic_0   Topic_1   Topic_2   Topic_3   Topic_4   Topic_5   Topic_6  \
game  0.880385 -0.071898 -0.036985 -0.012733 -0.074007 -0.005343 -0.000037   
bad   0.255875 -0.023508 -0.014813 -0.040111 -0.285748 -0.032076 -0.005643   
just  0.096241 -0.006920 -0.003053  0.007556  0.057866  0.006705  0.001465   
play  0.095117 -0.007097 -0.003071  0.006341  0.048784  0.005087  0.000872   
like  0.093150 -0.006698 -0.002989  0.006476  0.053850  0.006043  0.001225   

       Topic_7   Topic_8   Topic_9  abs_Topic_0  abs_Topic_1  abs_Topic_2  \
game -0.010643 -0.004916 -0.017098     0.880385     0.071898     0.036985   
bad  -0.200786 -0.118365  0.691203     0.255875     0.023508     0.014813   
just  0.041304  0.026731 -0.141512     0.096241     0.006920     0.003053   
play  0.035945  0.021410 -0.128346     0.095117     0.007097     0.003071   
like  0.035492  0.023779 -0.126311     0.093150     0.006698     0.002989   

      abs_Topic_3  abs_Topic_4  abs_Topic_5  abs_Topi

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e9ccb6fc-0232-4ed6-9cf8-093a8252a6f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>